In [ ]:
!pip install git+https://github.com/googleapis/python-aiplatform.git@mb-release

In [2]:
from google.cloud import aiplatform
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]
REGION='europe-west4'
BUCKET = "gs://"+PROJECT_ID

In [ ]:
VERSION='v1'
JOB_NAME="telco-churn-job-"+VERSION
MODEL_NAME="telco-churn-model-"+VERSION
DATASET_ID='5848592619210276864'

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET, location=REGION)

# ds = aiplatform.TabularDataset.create(display_name=display_name, gcs_source=gcs_source) # GCS dataset
# ds = aiplatform.TabularDataset.create(display_name=display_name, bigquery_source=bigquery_source) # BQ dataset

# ds.wait()


ds = aiplatform.Dataset('5848592619210276864') # For existing dataset! TODO: Change that to the created dataset id

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME, 
    python_package_gcs_uri=BUCKET+'/telco-churn/dist/trainer-0.1.tar.gz', 
    python_module_name='trainer.task', 
    container_uri='eu.gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest', 
    model_serving_container_image_uri='eu.gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest'
)

job.run(
    ds, replica_count=1, model_display_name=MODEL_NAME,
    base_output_dir=BUCKET+"/telco-churn/assets-"+VERSION,
    machine_type='n1-standard-4'
)

m = job.get_model()

endpoint = m.deploy(machine_type='n1-standard-4')

endpoint.predict([['Male', 0, True, False, True, False, 8, 20.25, '158.35', 'No', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'No internet service', 'Month-to-month', 'Mailed check', 'No'], ['Female', 0, False, False, True, True, 2, 87.15, '183.75', 'Fiber optic', 'No', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Bank transfer (automatic)', 'No']])